# **Preprocessing Data**

In [227]:
#import library
import pandas as pd
import numpy as np
import re
import ast
from haversine import haversine, Unit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib


In [228]:
#load data
data = ("../data/Dataset_Resto_Unair.csv")
df = pd.read_csv(data)

In [229]:
df.head()

,title,address,totalScore,categories,location
0,Bakso CITRA SATU PUTRA,"Jl. Raya Mulyosari No.177-179, Kalisari, Kec. ...",4.7,['Restoran Bakso'],"{'lat': -7.2707337, 'lng': 112.7969433}"
1,Bakso asli solo Pak Tono,"Jl. Gebang Lor No.49, Gebang Putih, Kec. Sukol...",4.5,['Restoran Bakso'],"{'lat': -7.2812233, 'lng': 112.7873977}"
2,Bakso Pak Anton,"Jl. Wisma Permai Tengah I No.CC-21, Kejawaan P...",4.8,"['Restoran Bakso', 'Rumah Makan']","{'lat': -7.2741205, 'lng': 112.7868231}"
3,Bakso Kepala Sapi,"Jl. Klampis Jaya No.47, Klampis Ngasem, Kec. S...",4.3,['Restoran Bakso'],"{'lat': -7.28232, 'lng': 112.7763487}"
4,Warung Bakso Dan Mie Wijaya,"Jl. Mulyorejo Tengah No.73, Mulyorejo, Kec. Mu...",4.5,"['Restoran Bakso', 'Restoran Mie']","{'lat': -7.2678847, 'lng': 112.7780222}"


In [230]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681 entries, 0 to 680
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       681 non-null    object 
 1   address     681 non-null    object 
 2   totalScore  681 non-null    float64
 3   categories  681 non-null    object 
 4   location    681 non-null    object 
dtypes: float64(1), object(4)
memory usage: 26.7+ KB


In [231]:
df.isnull().sum()

title         0
address       0
totalScore    0
categories    0
location      0
dtype: int64

In [232]:
df.duplicated().sum()

0

In [233]:
df = df.drop_duplicates()

In [234]:
df.describe()

,totalScore
count,681.000000
mean,4.569457
std,0.366551
min,1.800000
25%,4.400000
50%,4.600000
75%,4.800000
max,5.000000


# Preprocessing Data Categories

In [235]:
df['categories']

0                      ['Restoran Bakso']
1                      ['Restoran Bakso']
2       ['Restoran Bakso', 'Rumah Makan']
3                      ['Restoran Bakso']
4      ['Restoran Bakso', 'Restoran Mie']
                      ...                
676                              ['Kafe']
677                        ['Kedai Kopi']
678                        ['Kedai Kopi']
679                        ['Kedai Kopi']
680                        ['Kedai Kopi']
Name: categories, Length: 681, dtype: object

In [236]:
def remove_symbol(text):
    joined = ' '.join(text)
    cleaned_text = re.sub(r'[^A-Za-z\s]','',text)
    return cleaned_text

In [237]:
df['categories'] = df['categories'].apply(remove_symbol)

In [238]:
df['title'].tolist()

['Bakso CITRA SATU PUTRA',
 'Bakso asli solo Pak Tono',
 'Bakso Pak Anton',
 'Bakso Kepala Sapi',
 'Warung Bakso Dan Mie Wijaya',
 'Bakso & Mie Bondowoso',
 'Bakso Pangsit Mie Ayam Sensasi Rasa',
 'Bakso Dan Mie Ayam "Barokah"',
 'Bakso Solo Goyang Lidah (GOLI)',
 'Bakso Bem',
 'Bakso Solo Cak Awi',
 'Bakso jumbo sinar baru 2',
 'Bakso bratang Pindahan (ITATS)',
 'Warung LA',
 'Warung Kampus',
 'Warung Mbak Khol',
 'Warung Pak No Lamongan. ( Penyetan & Nasi Goreng )',
 'Warung Bagus',
 'Warung Makan Bu Yati',
 'Warung Sultan',
 'Warung Emak Murah Enak',
 'Warung Cak Nari Munos',
 'Warung Mbak Sieh',
 'Warung Penyetan Pak Kumis',
 'Warung Cak Di',
 'Warung Bu Warni Surabaya ( Nasi Petis-Specialist Cumi Hitam)',
 'Warung Satria',
 'Warung Mami 88',
 'Warung LA 2',
 'Bakso Daging Sapi Pak Sabar',
 'Bakso Bonnet',
 'Bakso Pak Djo',
 'Bakso Daging Sapi P.Sabar',
 'Bakso Pak War',
 'Bakso Pak Pek',
 'Mie Ayam & Bakso Solo',
 'Warung Tegal Gebang',
 'Warung Fadhila',
 'Warung Pak Hasyim',
 'W

# preprocessing data jarak

In [239]:
#Install library untuk hitung jarak dari kampus
!pip install haversine

You should consider upgrading via the 'C:\Users\LENOVO\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [240]:
df['location'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 681 entries, 0 to 680
Series name: location
Non-Null Count  Dtype 
--------------  ----- 
681 non-null    object
dtypes: object(1)
memory usage: 5.4+ KB


In [241]:
df['location'] = df['location'].apply(ast.literal_eval)

In [242]:
df['location'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 681 entries, 0 to 680
Series name: location
Non-Null Count  Dtype 
--------------  ----- 
681 non-null    object
dtypes: object(1)
memory usage: 5.4+ KB


In [243]:
#bujur dan lintang kampus a, b, dan c unair
kampus_a = (-7.2755,112.7634)
kampus_b = (-7.2713, 112.7688)
kampus_c = (-7.2697,112.7848)


In [244]:
#hitung jarak dalam satuan km dari kampus c
df['Jarak_KampusC'] = df['location'].apply(lambda x : haversine(kampus_c,( x['lat'], x['lng']), unit = Unit.KILOMETERS), 2)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_40608\2451219098.py:2: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df['Jarak_KampusC'] = df['location'].apply(lambda x : haversine(kampus_c,( x['lat'], x['lng']), unit = Unit.KILOMETERS), 2)


In [245]:
#hitung jarak dalam satuan km dari kampus b
df['Jarak_KampusB'] = df['location'].apply(lambda x : haversine(kampus_b,( x['lat'], x['lng']), unit = Unit.KILOMETERS),2)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_40608\284811816.py:2: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df['Jarak_KampusB'] = df['location'].apply(lambda x : haversine(kampus_b,( x['lat'], x['lng']), unit = Unit.KILOMETERS),2)


In [246]:
#hitung jarak dalam satuan km dari kampus a
df['Jarak_KampusA'] = df['location'].apply(lambda x : haversine(kampus_a,( x['lat'], x['lng']), unit = Unit.KILOMETERS),2)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_40608\3958307159.py:2: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df['Jarak_KampusA'] = df['location'].apply(lambda x : haversine(kampus_a,( x['lat'], x['lng']), unit = Unit.KILOMETERS),2)


In [247]:
df

,title,address,totalScore,categories,location,Jarak_KampusC,Jarak_KampusB,Jarak_KampusA
0,Bakso CITRA SATU PUTRA,"Jl. Raya Mulyosari No.177-179, Kalisari, Kec. ...",4.7,Restoran Bakso,"{'lat': -7.2707337, 'lng': 112.7969433}",1.344342,3.104870,3.737606
1,Bakso asli solo Pak Tono,"Jl. Gebang Lor No.49, Gebang Putih, Kec. Sukol...",4.5,Restoran Bakso,"{'lat': -7.2812233, 'lng': 112.7873977}",1.312979,2.329260,2.722356
2,Bakso Pak Anton,"Jl. Wisma Permai Tengah I No.CC-21, Kejawaan P...",4.8,Restoran Bakso Rumah Makan,"{'lat': -7.2741205, 'lng': 112.7868231}",0.539819,2.012544,2.588117
3,Bakso Kepala Sapi,"Jl. Klampis Jaya No.47, Klampis Ngasem, Kec. S...",4.3,Restoran Bakso,"{'lat': -7.28232, 'lng': 112.7763487}",1.684682,1.481480,1.617075
4,Warung Bakso Dan Mie Wijaya,"Jl. Mulyorejo Tengah No.73, Mulyorejo, Kec. Mu...",4.5,Restoran Bakso Restoran Mie,"{'lat': -7.2678847, 'lng': 112.7780222}",0.774372,1.085798,1.821619
...,...,...,...,...,...,...,...,...
676,WARKOP 3000,"Jl. Semolowaru No.44, Semolowaru, Kec. Sukolil...",4.4,Kafe,"{'lat': -7.3006098, 'lng': 112.7743827}",3.623989,3.316764,3.043539
677,Warkop 78 & TOKO IVA,"Depan parkiran kampus unair B, Jl. Srikana No....",5.0,Kedai Kopi,"{'lat': -7.2741453, 'lng': 112.7584342}",2.949874,1.186320,0.568064
678,Warkop R-11,"Jl. Semolowaru Elok No.11 blok R, Semolowaru, ...",4.8,Kedai Kopi,"{'lat': -7.307254, 'lng': 112.775239}",4.306918,4.060499,3.764608
679,Warkop DKN,"Jl. Medokan Semampir AWS No.22, RT.001/RW.006,...",4.6,Kedai Kopi,"{'lat': -7.3097995, 'lng': 112.7772405}",4.536154,4.381010,4.108095


In [248]:
df.to_csv('data_resto_with_jarak.csv', index= False)

In [249]:
df

,title,address,totalScore,categories,location,Jarak_KampusC,Jarak_KampusB,Jarak_KampusA
0,Bakso CITRA SATU PUTRA,"Jl. Raya Mulyosari No.177-179, Kalisari, Kec. ...",4.7,Restoran Bakso,"{'lat': -7.2707337, 'lng': 112.7969433}",1.344342,3.104870,3.737606
1,Bakso asli solo Pak Tono,"Jl. Gebang Lor No.49, Gebang Putih, Kec. Sukol...",4.5,Restoran Bakso,"{'lat': -7.2812233, 'lng': 112.7873977}",1.312979,2.329260,2.722356
2,Bakso Pak Anton,"Jl. Wisma Permai Tengah I No.CC-21, Kejawaan P...",4.8,Restoran Bakso Rumah Makan,"{'lat': -7.2741205, 'lng': 112.7868231}",0.539819,2.012544,2.588117
3,Bakso Kepala Sapi,"Jl. Klampis Jaya No.47, Klampis Ngasem, Kec. S...",4.3,Restoran Bakso,"{'lat': -7.28232, 'lng': 112.7763487}",1.684682,1.481480,1.617075
4,Warung Bakso Dan Mie Wijaya,"Jl. Mulyorejo Tengah No.73, Mulyorejo, Kec. Mu...",4.5,Restoran Bakso Restoran Mie,"{'lat': -7.2678847, 'lng': 112.7780222}",0.774372,1.085798,1.821619
...,...,...,...,...,...,...,...,...
676,WARKOP 3000,"Jl. Semolowaru No.44, Semolowaru, Kec. Sukolil...",4.4,Kafe,"{'lat': -7.3006098, 'lng': 112.7743827}",3.623989,3.316764,3.043539
677,Warkop 78 & TOKO IVA,"Depan parkiran kampus unair B, Jl. Srikana No....",5.0,Kedai Kopi,"{'lat': -7.2741453, 'lng': 112.7584342}",2.949874,1.186320,0.568064
678,Warkop R-11,"Jl. Semolowaru Elok No.11 blok R, Semolowaru, ...",4.8,Kedai Kopi,"{'lat': -7.307254, 'lng': 112.775239}",4.306918,4.060499,3.764608
679,Warkop DKN,"Jl. Medokan Semampir AWS No.22, RT.001/RW.006,...",4.6,Kedai Kopi,"{'lat': -7.3097995, 'lng': 112.7772405}",4.536154,4.381010,4.108095


# **Feature Extraction**

In [250]:
#tf idf untuk  sama catgories
data = df.copy()
tfidf = TfidfVectorizer()
data['combined'] = data['categories'] +' '+ data['title']
data['combined'] = data['combined'].apply(remove_symbol)
data['combined'] = data['combined'].str.strip().str.lower()


tfidf_categories = tfidf.fit_transform(data['combined'])


joblib.dump(tfidf, '../model/tfidf.pkl')

joblib.dump(tfidf_categories, '../model/tfidf_matrix.pkl')

['../model/tfidf_matrix.pkl']

In [251]:
data['combined']

0                  restoran bakso bakso citra satu putra
1                restoran bakso bakso asli solo pak tono
2             restoran bakso rumah makan bakso pak anton
3                       restoran bakso bakso kepala sapi
4      restoran bakso restoran mie warung bakso dan m...
                             ...                        
676                                          kafe warkop
677                         kedai kopi warkop   toko iva
678                                  kedai kopi warkop r
679                                kedai kopi warkop dkn
680                            kedai kopi warkop lestari
Name: combined, Length: 681, dtype: object

In [252]:
cosin = cosine_similarity(tfidf_categories)
cosin_df = pd.DataFrame(
    cosin,
    index= data['combined'],
    columns= data['combined']
)

cosin_df.index = cosin_df.index.str.strip().str.lower()
cosin_df.columns = cosin_df.columns.str.strip().str.lower()

joblib.dump(cosin_df, '../model/cosinesimilarity.pkl')

['../model/cosinesimilarity.pkl']

In [253]:
cosin_df.dtypes

combined
restoran bakso bakso citra satu putra                      float64
restoran bakso bakso asli solo pak tono                    float64
restoran bakso rumah makan bakso pak anton                 float64
restoran bakso bakso kepala sapi                           float64
restoran bakso restoran mie warung bakso dan mie wijaya    float64
                                                            ...   
kafe warkop                                                float64
kedai kopi warkop   toko iva                               float64
kedai kopi warkop r                                        float64
kedai kopi warkop dkn                                      float64
kedai kopi warkop lestari                                  float64
Length: 681, dtype: object

In [254]:
cosin_df.columns = cosin_df.columns.str.strip().str.lower()
cosin_df.index = cosin_df.index.str.strip().str.lower()

cosin_df.index

Index(['restoran bakso bakso citra satu putra',
       'restoran bakso bakso asli solo pak tono',
       'restoran bakso rumah makan bakso pak anton',
       'restoran bakso bakso kepala sapi',
       'restoran bakso restoran mie warung bakso dan mie wijaya',
       'restoran bakso restoran mie bakso  mie bondowoso',
       'restoran mie restoran bakso bakso pangsit mie ayam sensasi rasa',
       'restoran bakso restoran mie bakso dan mie ayam barokah',
       'restoran bakso restoran mie bakso solo goyang lidah goli',
       'restoran bakso bakso bem',
       ...
       'kedai kopi kafe warkop gresik semolowaru', 'kafe warkop gayam',
       'kafe seni warkop  safaraz', 'kedai kopi warkop adi',
       'kedai kopi warkop cak kasnun', 'kafe warkop',
       'kedai kopi warkop   toko iva', 'kedai kopi warkop r',
       'kedai kopi warkop dkn', 'kedai kopi warkop lestari'],
      dtype='object', name='combined', length=681)

In [255]:
def display_name(category_query, similarity_df, item_data,base,  k=5):
    category_query = category_query.strip().lower()
    
    # Cari kolom similarity yang cocok berdasarkan string
    matched_cols = [col for col in similarity_df.columns if category_query in col.lower()]
    
    if not matched_cols:
        print(f"Tidak ditemukan hasil untuk kategori: '{category_query}'")
        return pd.DataFrame()

    selected_col = matched_cols[0]
    similarity = similarity_df[selected_col]

    top_k = similarity.nlargest(k)
    top_k = top_k.groupby(level= 0).max()
    ## print(top_k)

    # Persiapan data
    item_data['categories_cleaned'] = item_data['categories'].apply(remove_symbol)
    item_data['title_cleaned'] = item_data['title'].apply(remove_symbol)
    item_data['combined'] = item_data['categories_cleaned'] + ' ' + item_data['title_cleaned']
    item_data['combined'] = item_data['combined'].str.strip().str.lower()
    
    recom = item_data[item_data['combined'].isin(top_k.index)].copy()

    recom['similarity'] = recom['combined'].map(top_k)

    #hitung jarak dari kampus mana
    base_column = f'Jarak_Kampus{base.upper()}'
    if base_column not in recom.columns:
        print(f"Masukkan dari UNAIR Kampus Mana")
    else:
        recom = recom.sort_values(by= [base_column], ascending= True)
    #ambil berdasarkan rating tertinggi dan similarity tertinggi 
    print(recom[['combined', 'totalScore', 'similarity']].head(5))
    return recom.sort_values(by=[f'{base_column}','totalScore', 'similarity'], ascending=[True, False, False])

    


In [256]:
display_name(
    category_query= 'bakso',
    similarity_df= cosin_df,
    item_data= df,
    base= 'A',
    k= 5
)



                                 combined  totalScore  similarity
66           restoran bakso bakso cak nur         4.3    0.382640
88           restoran bakso bakso bratang         4.4    0.401381
0   restoran bakso bakso citra satu putra         4.7    1.000000
61              restoran bakso bakso merr         4.5    0.401381
64             restoran bakso bakso unyil         4.5    0.393404


,title,address,totalScore,categories,location,Jarak_KampusC,Jarak_KampusB,Jarak_KampusA,categories_cleaned,title_cleaned,combined,similarity
66,Bakso Cak Nur 1,"Jl. Ngagel Jaya Tengah No.114, Baratajaya, Kec...",4.3,Restoran Bakso,"{'lat': -7.291864, 'lng': 112.7616669}",3.547434,2.418185,1.829610,Restoran Bakso,Bakso Cak Nur,restoran bakso bakso cak nur,0.382640
88,Bakso Bratang,"No 92, Jl. Barata Jaya XIX, Baratajaya, Kec. G...",4.4,Restoran Bakso,"{'lat': -7.2993486, 'lng': 112.7614064}",4.186464,3.223718,2.660948,Restoran Bakso,Bakso Bratang,restoran bakso bakso bratang,0.401381
0,Bakso CITRA SATU PUTRA,"Jl. Raya Mulyosari No.177-179, Kalisari, Kec. ...",4.7,Restoran Bakso,"{'lat': -7.2707337, 'lng': 112.7969433}",1.344342,3.104870,3.737606,Restoran Bakso,Bakso CITRA SATU PUTRA,restoran bakso bakso citra satu putra,1.000000
61,Bakso Merr99,"Jl. Semampir Tengah No.38, Medokan Semampir, K...",4.5,Restoran Bakso,"{'lat': -7.3072864, 'lng': 112.7815193}",4.195058,4.240306,4.060366,Restoran Bakso,Bakso Merr,restoran bakso bakso merr,0.401381
64,Bakso Unyil,"PR55+4R2, Keputih, Kec. Sukolilo, Surabaya, Ja...",4.5,Restoran Bakso,"{'lat': -7.292248, 'lng': 112.809519}",3.704026,5.059338,5.417001,Restoran Bakso,Bakso Unyil,restoran bakso bakso unyil,0.393404
